In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import csv
import time



# Loading original dataset

In [2]:
dataset = pd.read_csv('data4.csv', delimiter=";")



C:\Users\silva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# build timestamp

In [3]:
dataset.date = dataset.date+ ' ' +dataset.time
dataset.date= pd.to_datetime(dataset.date, infer_datetime_format=True)


# drop unused columns

In [4]:
dataset.drop('time', axis=1, inplace=True)


# selecting sensor dataset 

In [5]:
nset  = dataset[(dataset["moteid"] ==1)]
nset

,date,epoch,moteid,temperature,humidity,light,voltage,Unnamed: 8
0,2004-03-31 03:38:15.757551,2,1,122.153,-391.901,11.04,203.397,NaN
1,2004-02-28 00:59:16.027850,3,1,199.884,370.933,45.08,269.964,NaN
2,2004-02-28 01:03:16.333930,11,1,193.024,384.629,45.08,268.742,NaN
3,2004-02-28 01:06:16.013453,17,1,191.652,388.039,45.08,268.742,NaN
4,2004-02-28 01:06:46.778088,18,1,19.175,388.379,45.08,269.964,NaN
...,...,...,...,...,...,...,...,...
43042,2004-03-21 19:01:04.104563,65525,1,211.056,459.669,2.30,235.683,NaN
43043,2004-03-21 19:02:32.920740,65528,1,21.086,460.655,2.30,235.683,NaN
43044,2004-03-21 19:03:32.861656,65530,1,21.086,461.968,2.30,235.683,NaN
43045,2004-03-21 19:04:56.432522,65533,1,210.762,460.326,0.92,235.683,NaN


# build ML dataset part 1

In [6]:
MyLIst = []

for i in np.arange((nset.shape[0]-1)):
        dalay = (nset.iloc[i+1, 0] - nset.iloc[i, 0]).total_seconds()
        delta_temperature = abs(nset.iloc[i+1, 3] - nset.iloc[i, 3])
        sensor_id = nset.iloc[i,2]
        hour = nset.iloc[i, 0].hour
        minute = nset.iloc[i, 0].minute
        second = nset.iloc[i, 0].second
        day = nset.iloc[i, 0].day
        month = nset.iloc[i, 0].month
        label = 0
        if(delta_temperature>=3):
            label=1
        MyLIst.append([nset.iloc[i, 0], hour,minute,second,day,month, dalay, sensor_id, nset.iloc[i, 3], label])
    


In [7]:
cols = ['date', 'hour','minut','second','day','month', 'delay', 'sensor_id', 'temperature','label']
df_ML = pd.DataFrame(MyLIst, columns = cols)


# updating label and dalay to concept drift

In [8]:
def create_window(X, time_steps=50):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i: (i+time_steps), 8].to_numpy()
        Xs.append(v)
    return np.array(Xs)

In [9]:
df_ML.iloc[:,8].mean()

184.22567283834934

In [18]:
WINDOW = 50
data_stream = create_window(df_ML, time_steps=WINDOW)
data_stream

array([[122.153, 199.884, 193.024, ..., 189.006, 189.104, 189.006],
       [199.884, 193.024, 191.652, ..., 189.104, 189.006, 188.908],
       [193.024, 191.652,  19.175, ..., 189.006, 188.908, 189.006],
       ...,
       [217.426,  21.723,  21.723, ..., 211.448,  21.135, 211.056],
       [ 21.723,  21.723, 216.838, ...,  21.135, 211.056,  21.086],
       [ 21.723, 216.838, 216.838, ..., 211.056,  21.086,  21.086]])

In [19]:
# Imports
import numpy as np
from skmultiflow.drift_detection import PageHinkley
ph = PageHinkley()

# Adding stream elements to the PageHinkley drift detector and verifying if drift occurred
CHANGE = 0
DELAY= 0 
for i in np.arange(data_stream.shape[0]):  
    print(data_stream[i])
    ph.add_element(data_stream[i])
    if ph.detected_change():
        CHANGE = 1
        print('Change has been detected in data: ' + str(data_stream[i]) + 'delay'+str(DELAY) +' - of index: '+str(i) )
    init = i*WINDOW    
    df_ML.iloc[init: (init+WINDOW), 9] = CHANGE
    df_ML.iloc[init: (init+WINDOW), 6] =  dalay
    DELAY +=200
    if(CHANGE):
        CHANGE = 0
    DELAY= 0 
        
    

[122.153 199.884 193.024 191.652  19.175 191.456 191.652 191.652 191.456
 191.456 191.456 191.358 191.162 191.162 191.064 191.064 190.966 190.966
 190.868 190.672 190.672 190.672 190.868 190.182 190.182 190.084 190.084
 190.084 189.986 190.084 189.888 189.692  18.979  18.979 189.692 189.888
  18.979 189.888 189.692 189.594 189.594 189.692 189.398  18.93  189.202
 189.104 189.006 189.006 189.104 189.006]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [11]:
data_stream[0]

array([122.153, 199.884, 193.024, 191.652,  19.175, 191.456, 191.652,
       191.652, 191.456, 191.456, 191.456, 191.358, 191.162, 191.162,
       191.064, 191.064, 190.966, 190.966, 190.868, 190.672, 190.672,
       190.672, 190.868, 190.182, 190.182, 190.084, 190.084, 190.084,
       189.986, 190.084, 189.888, 189.692,  18.979,  18.979, 189.692,
       189.888,  18.979, 189.888, 189.692, 189.594, 189.594, 189.692,
       189.398,  18.93 , 189.202, 189.104, 189.006, 189.006, 189.104,
       189.006])

In [ ]:
df_ML

In [ ]:
f = open('sdn_ml.csv','w')
try:
    writer = csv.writer(f)
    writer.writerow(df_ML.columns)
    for i in np.arange(int(df_ML.shape[0])):
        writer.writerow(df_ML.iloc[i,])
finally:
    f.close()

In [ ]:
# MyLIst = []
# for i in np.arange((nset.shape[0]-1)):
#     if(nset.iloc[i+1, 2] == nset.iloc[i, 2]):
#         dalay = (nset.iloc[i+1, 0] - nset.iloc[i, 0]).total_seconds()
#         temperature = nset.iloc[i+1, 3] - nset.iloc[i, 3]
#         label = 0
#         if(temperature>=3):
#             label=1
#         MyLIst.append([time.mktime(nset.iloc[i, 0].timetuple()), nset.iloc[i, 0], dalay, nset.iloc[i,2], nset.iloc[i, 3], label])
    
